In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.obt
USING DELTA
PARTITIONED BY (run_time)
AS
SELECT
  cr.*,
  dm.sop,
  dm.method_id,
  dm.meth_id,
  dm.temp_id,
  dm.report_template_version,
  dm.seq_id,
  dm.seq_version,
  ds.equipment_id,
  ds.im_id,
  ds.column_id,
  du.submit_id,
  du.submit_datetime,
  du.review_id,
  du.review_datetime,
  du.approval_id,
  du.approval_datetime,
  du.status
FROM workspace.gold_serving.run_sst_result cr
LEFT JOIN workspace.gold_serving.dimension_methods dm
  ON cr.pk = dm.method_id OR cr.run_time = dm.time
LEFT JOIN workspace.gold_serving.dimension_systems ds
  ON cr.run_time = ds.time
LEFT JOIN workspace.gold_serving.dimension_users du
  ON cr.run_time = du.time;


MERGE INTO workspace.gold_serving.obt AS target
USING (
  SELECT * FROM (
    SELECT
      cr.*,
      dm.sop,
      dm.method_id,
      dm.meth_id,
      dm.temp_id,
      dm.report_template_version,
      dm.seq_id,
      dm.seq_version,
      ds.equipment_id,
      ds.im_id,
      ds.column_id,
      du.submit_id,
      du.submit_datetime,
      du.review_id,
      du.review_datetime,
      du.approval_id,
      du.approval_datetime,
      du.status,
      ROW_NUMBER() OVER (
        PARTITION BY cr.pk, cr.samp_id, cr.sst_number
        ORDER BY cr.run_time DESC
      ) AS rn
    FROM workspace.gold_serving.run_sst_result cr
    LEFT JOIN workspace.gold_serving.dimension_methods dm
      ON cr.pk = dm.method_id OR cr.run_time = dm.time
    LEFT JOIN workspace.gold_serving.dimension_systems ds
      ON cr.run_time = ds.time
    LEFT JOIN workspace.gold_serving.dimension_users du
      ON cr.run_time = du.time
  ) sub
  WHERE rn = 1
) AS source
ON target.pk = source.pk
AND target.run_time = source.run_time
AND target.samp_id = source.samp_id
AND target.sst_number = source.sst_number

WHEN MATCHED THEN
  UPDATE SET
    target.run_time = source.run_time,
    target.in_num = source.in_num,
    target.in_name = source.in_name,
    target.pk = source.pk,
    target.sst_number = source.sst_number,
    target.sst_in_num = source.sst_in_num,
    target.sst_in_name = source.sst_in_name,
    target.sst_name = source.sst_name,
    target.peak = source.peak,
    target.eval_result = source.eval_result,
    target.sst_result = source.sst_result,
    target.samp_id = source.samp_id,
    target.as_id = source.as_id,
    target.Type = source.Type,
    target.comp = source.comp,
    target.unit = source.unit,
    target.det_id = source.det_id,
    target.assay_result = source.assay_result,
    target.number_of_averaged_samples = source.number_of_averaged_samples,
    target.sop = source.sop,
    target.method_id = source.method_id,
    target.meth_id = source.meth_id,
    target.temp_id = source.temp_id,
    target.report_template_version = source.report_template_version,
    target.seq_id = source.seq_id,
    target.seq_version = source.seq_version,
    target.equipment_id = source.equipment_id,
    target.im_id = source.im_id,
    target.column_id = source.column_id,
    target.submit_id = source.submit_id,
    target.submit_datetime = source.submit_datetime,
    target.review_id = source.review_id,
    target.review_datetime = source.review_datetime,
    target.approval_id = source.approval_id,
    target.approval_datetime = source.approval_datetime,
    target.status = source.status

WHEN NOT MATCHED THEN
  INSERT (
    run_time,
    in_num,
    in_name,
    pk,
    sst_number,
    sst_in_num,
    sst_in_name,
    sst_name,
    peak,
    eval_result,
    sst_result,
    samp_id,
    as_id,
    Type,
    comp,
    unit,
    det_id,
    assay_result,
    number_of_averaged_samples,
    sop,
    method_id,
    meth_id,
    temp_id,
    report_template_version,
    seq_id,
    seq_version,
    equipment_id,
    im_id,
    column_id,
    submit_id,
    submit_datetime,
    review_id,
    review_datetime,
    approval_id,
    approval_datetime,
    status
  )
  VALUES (
    source.run_time,
    source.in_num,
    source.in_name,
    source.pk,
    source.sst_number,
    source.sst_in_num,
    source.sst_in_name,
    source.sst_name,
    source.peak,
    source.eval_result,
    source.sst_result,
    source.samp_id,
    source.as_id,
    source.Type,
    source.comp,
    source.unit,
    source.det_id,
    source.assay_result,
    source.number_of_averaged_samples,
    source.sop,
    source.method_id,
    source.meth_id,
    source.temp_id,
    source.report_template_version,
    source.seq_id,
    source.seq_version,
    source.equipment_id,
    source.im_id,
    source.column_id,
    source.submit_id,
    source.submit_datetime,
    source.review_id,
    source.review_datetime,
    source.approval_id,
    source.approval_datetime,
    source.status
  );


